<a href="https://colab.research.google.com/github/tdang2180/Mobility-Scooter-Speed-Calculation/blob/main/Mobility_Scooter_Speed_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
df = pd.read_csv('/April_5_Run_1.csv')
df['acc_magnitude'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)
df['gyro_magnitude'] = np.sqrt(df['gyro_x']**2 + df['gyro_y']**2 + df['gyro_z']**2)
df['datetime'] = pd.to_datetime(df['datetime'])
df['time'] = df['datetime'].dt.time
print(df[['acc_x','acc_y','acc_z', 'acc_magnitude', 'time']])

         acc_x     acc_y     acc_z  acc_magnitude             time
0     0.064976 -0.735980  0.675528       1.001113  11:33:45.837070
1     0.067158 -0.693042  0.677234       0.971320  11:33:46.364798
2     0.068855 -0.695454  0.677965       0.973670  11:33:46.915594
3     0.067885 -0.693042  0.680645       0.973752  11:33:47.466455
4     0.067642 -0.693042  0.680889       0.973906  11:33:48.001725
...        ...       ...       ...            ...              ...
2260  0.056732 -0.692560  0.697217       0.984362  11:56:02.973306
2261  0.057460 -0.691353  0.696973       0.983383  11:56:03.570811
2262  0.056975 -0.692318  0.700872       0.986799  11:56:04.164838
2263  0.056732 -0.693283  0.695267       0.983492  11:56:04.747821
2264  0.057217 -0.691353  0.696242       0.982851  11:56:05.342520

[2265 rows x 5 columns]


In [30]:
# Use linear interpolation to estimate values of speed at unrecorded times (every second)
def interpolate_speed(s):
    timestamps = pd.date_range(start=df['datetime'].iloc[0], end=df['datetime'].iloc[-1], freq='1s')
    df_interp = pd.DataFrame({'datetime': timestamps})
    df_interp['speed'] = np.interp(df_interp['datetime'], df['datetime'], df['speed'])
    df_interp.to_csv(f"{s}_interpolate.csv")
    print(df_interp)

In [59]:
# Use the magnitude of acceleration to find the magnitude of speed
def acc_magnitude_speed():
    df['speed'] =(df['acc_magnitude'] * df['datetime'].diff())
    df['speed'] = df['speed'].dt.total_seconds()  # Convert timedelta to seconds
    df['speed'] = df['speed'].fillna(0).cumsum()
    df[['time', 'acc_magnitude', 'speed']].to_csv("acc_magnitude_speed.csv")
    print(df[['time', 'acc_magnitude', 'speed']])

In [60]:
acc_magnitude_speed()

                 time  acc_magnitude        speed
0     11:33:45.837070       1.001113     0.000000
1     11:33:46.364798       0.971320     0.512593
2     11:33:46.915594       0.973670     1.048886
3     11:33:47.466455       0.973752     1.585289
4     11:33:48.001725       0.973906     2.106591
...               ...            ...          ...
2260  11:56:02.973306       0.984362  1306.375739
2261  11:56:03.570811       0.983383  1306.963315
2262  11:56:04.164838       0.986799  1307.549501
2263  11:56:04.747821       0.983492  1308.122860
2264  11:56:05.342520       0.982851  1308.707360

[2265 rows x 3 columns]


In [35]:
interpolate_speed("acc_magnitude_speed")

                       datetime        speed
0    2023-04-04 11:33:45.837070     0.000000
1    2023-04-04 11:33:46.837070     0.972430
2    2023-04-04 11:33:47.837070     1.946233
3    2023-04-04 11:33:48.837070     2.921756
4    2023-04-04 11:33:49.837070     3.897001
...                         ...          ...
1335 2023-04-04 11:56:00.837070  1304.273549
1336 2023-04-04 11:56:01.837070  1305.257072
1337 2023-04-04 11:56:02.837070  1306.241633
1338 2023-04-04 11:56:03.837070  1307.226060
1339 2023-04-04 11:56:04.837070  1308.210578

[1340 rows x 2 columns]


In [36]:
# Calculate velocity vectors before finding the magnitude of speed
def velocity_speed():
    df['velocity_x'] = (df['acc_x'] * df['datetime'].diff())
    df['velocity_x'] = df['velocity_x'].dt.total_seconds()  # Convert timedelta to seconds
    df['velocity_x'] = df['velocity_x'].fillna(0).cumsum()

    df['velocity_y'] = (df['acc_y'] * df['datetime'].diff())
    df['velocity_y'] = df['velocity_y'].dt.total_seconds()  # Convert timedelta to seconds
    df['velocity_y'] = df['velocity_y'].fillna(0).cumsum()

    df['velocity_z'] = (df['acc_z'] * df['datetime'].diff())
    df['velocity_z'] = df['velocity_z'].dt.total_seconds()  # Convert timedelta to seconds
    df['velocity_z'] = df['velocity_z'].fillna(0).cumsum()
    df['speed'] = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2 + df['velocity_z']**2)

    df[['time', 'velocity_x', 'velocity_y', 'velocity_z', 'speed']].to_csv("velocity_speed.csv")
    print(df[['time', 'velocity_x', 'velocity_y', 'velocity_z', 'speed']])

In [48]:
velocity_speed()

                 time  velocity_x  velocity_y  velocity_z        speed
0     11:33:45.837070    0.000000    0.000000    0.000000     0.000000
1     11:33:46.364798    0.035441   -0.365738    0.357395     0.512593
2     11:33:46.915594    0.073366   -0.748791    0.730815     1.048886
3     11:33:47.466455    0.110761   -1.130561    1.105756     1.585286
4     11:33:48.001725    0.146968   -1.501526    1.470216     2.106588
...               ...         ...         ...         ...          ...
2260  11:56:02.973306   74.498281 -920.200693  913.708906  1298.916192
2261  11:56:03.570811   74.532614 -920.613780  914.125351  1299.503751
2262  11:56:04.164838   74.566458 -921.025036  914.541688  1300.089909
2263  11:56:04.747821   74.599532 -921.429208  914.947017  1300.663261
2264  11:56:05.342520   74.633559 -921.840355  915.361071  1301.247747

[2265 rows x 5 columns]


In [38]:
interpolate_speed("velocity_speed")

                       datetime        speed
0    2023-04-04 11:33:45.837070     0.000000
1    2023-04-04 11:33:46.837070     0.972430
2    2023-04-04 11:33:47.837070     1.946230
3    2023-04-04 11:33:48.837070     2.921747
4    2023-04-04 11:33:49.837070     3.896990
...                         ...          ...
1335 2023-04-04 11:56:00.837070  1296.814066
1336 2023-04-04 11:56:01.837070  1297.797559
1337 2023-04-04 11:56:02.837070  1298.782090
1338 2023-04-04 11:56:03.837070  1299.766483
1339 2023-04-04 11:56:04.837070  1300.750977

[1340 rows x 2 columns]


In [51]:
# Use the magnitude of acceleration and time elapsed points to find the magnitude of speed
def acc_magnitude_speed_elapsed():
    df['speed'] =(df['acc_magnitude'] * df['elapsed'].diff()).fillna(df['acc_magnitude'].iloc[0] * df['elapsed'].iloc[0])
    df['speed'] = df['speed'].fillna(0).cumsum()
    df[['time', 'acc_magnitude', 'elapsed', 'speed']].to_csv("acc_magnitude_speed_elapsed.csv")
    print(df[['time', 'acc_magnitude', 'elapsed', 'speed']])

In [52]:
acc_magnitude_speed_elapsed()

                 time  acc_magnitude  elapsed        speed
0     11:33:45.837070       1.001113      6.1     6.106787
1     11:33:46.364798       0.971320      6.6     6.592447
2     11:33:46.915594       0.973670      7.2     7.176649
3     11:33:47.466455       0.973752      7.7     7.663525
4     11:33:48.001725       0.973906      8.2     8.150478
...               ...            ...      ...          ...
2260  11:56:02.973306       0.984362   1343.2  1312.433036
2261  11:56:03.570811       0.983383   1343.8  1313.023066
2262  11:56:04.164838       0.986799   1344.4  1313.615145
2263  11:56:04.747821       0.983492   1345.0  1314.205240
2264  11:56:05.342520       0.982851   1345.6  1314.794951

[2265 rows x 4 columns]


In [53]:
# Calculate velocity vectors before finding the magnitude of speed using time elasped points
def velocity_speed_elapsed():
    df['velocity_x'] = (df['acc_x'] * df['elapsed'].diff()).fillna(df['acc_x'].iloc[0] * df['elapsed'].iloc[0])
    df['velocity_x'] = df['velocity_x'].fillna(0).cumsum()

    df['velocity_y'] = (df['acc_y'] * df['elapsed'].diff()).fillna(df['acc_y'].iloc[0] * df['elapsed'].iloc[0])
    df['velocity_y'] = df['velocity_y'].fillna(0).cumsum()

    df['velocity_z'] = (df['acc_z'] * df['elapsed'].diff()).fillna(df['acc_z'].iloc[0] * df['elapsed'].iloc[0])
    df['velocity_z'] = df['velocity_z'].fillna(0).cumsum()
    df['speed'] = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2 + df['velocity_z']**2)

    df[['time', 'velocity_x', 'velocity_y', 'velocity_z', 'elapsed', 'speed']].to_csv("velocity_speed_elapsed.csv")
    print(df[['time', 'velocity_x', 'velocity_y', 'velocity_z', 'elapsed', 'speed']])

In [54]:
velocity_speed_elapsed()

                 time  velocity_x  velocity_y  velocity_z  elapsed  \
0     11:33:45.837070    0.396351   -4.489479    4.120719      6.1   
1     11:33:46.364798    0.429929   -4.836000    4.459336      6.6   
2     11:33:46.915594    0.471242   -5.253273    4.866114      7.2   
3     11:33:47.466455    0.505185   -5.599794    5.206437      7.7   
4     11:33:48.001725    0.539006   -5.946315    5.546882      8.2   
...               ...         ...         ...         ...      ...   
2260  11:56:02.973306   74.859559 -924.643035  917.758335   1343.2   
2261  11:56:03.570811   74.894035 -925.057847  918.176519   1343.8   
2262  11:56:04.164838   74.928220 -925.473238  918.597042   1344.4   
2263  11:56:04.747821   74.962259 -925.889208  919.014203   1345.0   
2264  11:56:05.342520   74.996590 -926.304020  919.431948   1345.6   

            speed  
0        6.106787  
1        6.592224  
2        7.176212  
3        7.662890  
4        8.149668  
...           ...  
2260  1304.932588  

In [55]:
def acc_difference_speed_elapsed():
    df['speed'] =(df['acc_magnitude'].diff() * df['elapsed'].diff()).fillna(df['acc_magnitude'].iloc[0] * df['elapsed'].iloc[0])
    df['speed'] = df['speed'].fillna(0).cumsum()
    df[['time', 'acc_magnitude', 'elapsed', 'speed']].to_csv("acc_difference_speed_elapsed.csv")
    print(df[['time', 'acc_magnitude', 'elapsed', 'speed']])

In [56]:
acc_difference_speed_elapsed()

                 time  acc_magnitude  elapsed     speed
0     11:33:45.837070       1.001113      6.1  6.106787
1     11:33:46.364798       0.971320      6.6  6.091890
2     11:33:46.915594       0.973670      7.2  6.093300
3     11:33:47.466455       0.973752      7.7  6.093342
4     11:33:48.001725       0.973906      8.2  6.093418
...               ...            ...      ...       ...
2260  11:56:02.973306       0.984362   1343.2  5.971293
2261  11:56:03.570811       0.983383   1343.8  5.970706
2262  11:56:04.164838       0.986799   1344.4  5.972755
2263  11:56:04.747821       0.983492   1345.0  5.970771
2264  11:56:05.342520       0.982851   1345.6  5.970387

[2265 rows x 4 columns]
